In [1]:
# modelling
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

import pickle

parent_dir = '../'

Using TensorFlow backend.


In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from src import modelling as m

Import the model

In [3]:
model5 = load_model(parent_dir + 'models/model5_old4', custom_objects = None, compile = True)

Get file path to static demo images

In [4]:
demo_images_dir = parent_dir + 'deployment/static/images'

Create data generator

In [5]:
# All images will be rescaled by 1./255
demo_datagen = ImageDataGenerator(rescale=1./255)

demo_generator = demo_datagen.flow_from_directory(demo_images_dir, target_size = (256, 256),
                                   batch_size = 32, class_mode = None,
                                   shuffle = False)

Found 30 images belonging to 2 classes.


Get array of predictions

In [7]:
preds = m.get_preds(model5, demo_generator)

In [8]:
preds

array([2, 3, 3, 4, 2, 5, 1, 1, 2, 4, 1, 1, 5, 1, 0, 3, 5, 3, 0, 3, 2, 5,
       3, 2, 4, 0, 2, 3, 0, 0])

Assign filepaths to images 

In [38]:
images = demo_generator.filepaths

In [39]:
images

['../deployment/static/images/imgs/1162.jpg',
 '../deployment/static/images/imgs/119.jpg',
 '../deployment/static/images/imgs/1223.jpg',
 '../deployment/static/images/imgs/1268.jpg',
 '../deployment/static/images/imgs/1404.jpg',
 '../deployment/static/images/imgs/145.jpg',
 '../deployment/static/images/imgs/1524.jpg',
 '../deployment/static/images/imgs/1704.jpg',
 '../deployment/static/images/imgs/176.jpg',
 '../deployment/static/images/imgs/1834.jpg',
 '../deployment/static/images/imgs/1971.jpg',
 '../deployment/static/images/imgs/199.jpg',
 '../deployment/static/images/imgs/2016.jpg',
 '../deployment/static/images/imgs/2060.jpg',
 '../deployment/static/images/imgs/242.jpg',
 '../deployment/static/images/imgs/255.jpg',
 '../deployment/static/images/imgs/284.jpg',
 '../deployment/static/images/imgs/319.jpg',
 '../deployment/static/images/imgs/350.jpg',
 '../deployment/static/images/imgs/365.jpg',
 '../deployment/static/images/imgs/38.jpg',
 '../deployment/static/images/imgs/3830.jpg',


The app can only take images with filepaths starting with '/static/' so we need to strip the start of the filepath:
_____

Test first:

In [22]:
test = '../deployment/static/images'

In [23]:
test = test.strip('../deployment')

In [24]:
test

'static/images'

_____

In [40]:
images = [i.replace('../deployment', '') for i in images]
images

['/static/images/imgs/1162.jpg',
 '/static/images/imgs/119.jpg',
 '/static/images/imgs/1223.jpg',
 '/static/images/imgs/1268.jpg',
 '/static/images/imgs/1404.jpg',
 '/static/images/imgs/145.jpg',
 '/static/images/imgs/1524.jpg',
 '/static/images/imgs/1704.jpg',
 '/static/images/imgs/176.jpg',
 '/static/images/imgs/1834.jpg',
 '/static/images/imgs/1971.jpg',
 '/static/images/imgs/199.jpg',
 '/static/images/imgs/2016.jpg',
 '/static/images/imgs/2060.jpg',
 '/static/images/imgs/242.jpg',
 '/static/images/imgs/255.jpg',
 '/static/images/imgs/284.jpg',
 '/static/images/imgs/319.jpg',
 '/static/images/imgs/350.jpg',
 '/static/images/imgs/365.jpg',
 '/static/images/imgs/38.jpg',
 '/static/images/imgs/3830.jpg',
 '/static/images/imgs/399.jpg',
 '/static/images/imgs/408.jpg',
 '/static/images/imgs/485.jpg',
 '/static/images/imgs/520.jpg',
 '/static/images/imgs/561.jpg',
 '/static/images/imgs/600.jpg',
 '/static/images/imgs/645.jpg',
 '/static/images/imgs/837.jpg']

Now we need to match up each image with its prediction.

In [55]:
paths_preds = zip(images, preds)

Create an albums dictionary which will store the images in their predicted class.

_______
don't need, don't run:

In [52]:
albums = {'Buildings': [], 'Forest': [], 'Glaciers': [], 
          'Mountains': [], 'Sea': [], 'Streets': []}

In [53]:
albums

{'Buildings': [],
 'Forest': [],
 'Glaciers': [],
 'Mountains': [],
 'Sea': [],
 'Streets': []}

____

Now, assign each image to its predicted class.

In [49]:
classes = {0: 'Buildings', 1: 'Forest', 2: 'Glaciers', 3: 'Mountains', 4: 'Sea', 5: 'Streets'}

In [57]:
albums = {}
for image, pred in paths_preds:
    if classes[pred] not in albums:
        albums[classes[pred]] = []
    albums[classes[pred]].append(image)
    
albums

{'Mountains': ['/static/images/imgs/119.jpg',
  '/static/images/imgs/1223.jpg',
  '/static/images/imgs/255.jpg',
  '/static/images/imgs/319.jpg',
  '/static/images/imgs/365.jpg',
  '/static/images/imgs/399.jpg',
  '/static/images/imgs/600.jpg'],
 'Sea': ['/static/images/imgs/1268.jpg',
  '/static/images/imgs/1834.jpg',
  '/static/images/imgs/485.jpg'],
 'Glaciers': ['/static/images/imgs/1404.jpg',
  '/static/images/imgs/176.jpg',
  '/static/images/imgs/38.jpg',
  '/static/images/imgs/408.jpg',
  '/static/images/imgs/561.jpg'],
 'Streets': ['/static/images/imgs/145.jpg',
  '/static/images/imgs/2016.jpg',
  '/static/images/imgs/284.jpg',
  '/static/images/imgs/3830.jpg'],
 'Forest': ['/static/images/imgs/1524.jpg',
  '/static/images/imgs/1704.jpg',
  '/static/images/imgs/1971.jpg',
  '/static/images/imgs/199.jpg',
  '/static/images/imgs/2060.jpg'],
 'Buildings': ['/static/images/imgs/242.jpg',
  '/static/images/imgs/350.jpg',
  '/static/images/imgs/520.jpg',
  '/static/images/imgs/645.jp

Now, pickle this dictionary so we can use it in our app. 

In [58]:
with open('predictions/demo_preds.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(albums, f, pickle.HIGHEST_PROTOCOL)